In [3]:
# Convert csv to geojson

import pandas as pd
import geojson

print(contour.columns)

Index(['id', 'annidx', 'elementidx', 'type', 'group', 'score', 'color', 'xmin',
       'ymin', 'xmax', 'ymax', 'bbox_area', 'coords_x', 'coords_y', 'keep'],
      dtype='object')


In [4]:
df = pd.read_csv("/home/sul084/immune-decoder/segmentation/NuHTC/demo/wsi_infer/TCGA-3C-AALI-01Z-00-DX2.CF4496E0-AB52-4F3E-BDF5-C34833B91B7C/contour_256_v2.csv")

# Prepare features
features = []
for _, row in df.iterrows():
    # Parse coords_x and coords_y, assuming they are comma-separated strings
    try:
        x_coords = list(map(float, row['coords_x'].split(',')))
        y_coords = list(map(float, row['coords_y'].split(',')))
    except Exception as e:
        print(f"Skipping row due to parsing error: {e}")
        continue

    # Zip into coordinate pairs
    if len(x_coords) != len(y_coords):
        print("Skipping row: mismatched coords")
        continue

    coords = list(zip(x_coords, y_coords))

    # Ensure polygon is closed
    if coords[0] != coords[-1]:
        coords.append(coords[0])

    polygon = geojson.Polygon([coords])

    # Drop geometry columns from properties
    props = row.drop(['coords_x', 'coords_y']).to_dict()

    features.append(geojson.Feature(geometry=polygon, properties=props))

# Create FeatureCollection
feature_collection = geojson.FeatureCollection(features)

# Save GeoJSON
with open("/home/sul084/immune-decoder/segmentation/NuHTC/demo/wsi_infer/TCGA-3C-AALI-01Z-00-DX2.CF4496E0-AB52-4F3E-BDF5-C34833B91B7C/contour_256_v2.geojson", "w") as f:
    geojson.dump(feature_collection, f)


In [2]:
!pip install geojson